# Forecasting

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

## Voorbeeld

In [ ]:
opbrengsten = [20,100,175,13,37,136,245,26,75,155,326,48,92,202,384,82,176,282,445,181]

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, 'o-') # wat doet 'o-'?
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Opbrengsten voorbije 5 jaar")
# fig.show()

## Naiëve forecasting

In [ ]:
def naive_forecasting(past):
  if len(past) < 1:
      return math.nan
  return past[-1]

In [ ]:
naive_forecasting(opbrengsten)

In [ ]:
# meerdere voorspellingen
verleden = opbrengsten.copy() # zonder copy wijzen verleden en opbrengsten naar hetzelfde object
voorspeld = [ ]
for i in range(4):
  volgende = naive_forecasting(verleden)
  voorspeld.append(volgende)
  verleden.append(volgende)
print("voorspelde waarden: ", voorspeld)

In [ ]:
# meer generiek:
def general_forecast(forecast_function, past, n):
    p = past.copy()
    result = []
    for i in range(n):
        next = forecast_function(p)
        result.append(next)
        p.append(next)
    return result

In [ ]:
voorspeld = general_forecast(naive_forecasting, opbrengsten, 4)
print(opbrengsten)
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20, 24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Naïeve forecasting")
# fig.show()

## Gemiddelde van alle waarden

In [ ]:
def average_forecasting(past):
  if len(past) < 1:
      return math.nan
  return pd.Series(past).mean()

In [ ]:
voorspeld = general_forecast(average_forecasting, opbrengsten, 4)
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20, 24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Gemiddelde forecasting")
# fig.show()

## Voortschrijdend gemiddelde

In [ ]:
def moving_average_forecasting(past, m):
    n = len(past)
    if n < m:
        return math.nan
    return pd.Series(past[-m:]).mean()

In [ ]:
moving_average_forecasting(verleden, 4)

In [ ]:
#voorspeld = general_forecast(moving_average_forecasting, verleden, 4) # werkt niet...  Waarom?
#print(voorspeld)

met extra functie:

In [ ]:
def moving_average_forecasting(m):
    def result(past):
        n = len(past)
        if n < m:
            return math.nan
        return pd.Series(past[-m:]).mean()
    return result

In [ ]:
forecast = moving_average_forecasting(4)
forecast(opbrengsten)

In [ ]:
voorspeld = general_forecast(forecast, opbrengsten, 4)
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20, 24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Voortschrijdende forecasting")
# fig.show()

## Lineaire combinatie

eerst gewichten berekenen:

In [ ]:
def calculate_weights(past, m):
  n = len(past)
  if n < 2*m:
      return m.nan
  v = past[(-2*m):-m]
  for i in range(2, m + 1):
    v = v + past[(-2*m+i-1):(-m+i-1)]
  M = np.array(v).reshape(m, m)
  v = past[-m:]
  return np.linalg.solve(M, v)

In [ ]:
a = calculate_weights(opbrengsten, 4)
print(a)

nu kunnen we voorspellen:

In [ ]:
periode = 4
n = len(opbrengsten)
voorspelling = (opbrengsten[-periode:]*a).sum()
print(voorspelling)

In [ ]:
# nu met functie
def lineair_combination_forecasting(m):
  def result(past):
    n = len(past)
    if n < 2*m:
        return math.nan
    a = calculate_weights(past, m)
    return (past[-m:]*a).sum()
  return result

In [ ]:
forecast = lineair_combination_forecasting(4)
forecast(opbrengsten)

In [ ]:
voorspeld = general_forecast(forecast, opbrengsten, 4)
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20, 24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Lineaire combinatie forecasting")
# fig.show()

## Betrouwbaarheid

In [ ]:
def forecast_past(past, forecast_function):
  result = []
  n = len(past)
  for i in range(n):
    result = result + [forecast_function(past[:i])]
  return result

In [ ]:
voorspeld = forecast_past(opbrengsten, naive_forecasting)
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
plt.plot(range(20), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Naïeve forecasting")
# fig.show()

MAE, RMSE en MAPE berekenen:

In [ ]:
voorspeld = forecast_past(opbrengsten, naive_forecasting)
fouten = pd.Series(voorspeld) - opbrengsten # waarom maken we eerst een pd.Series()?
MAE = fouten.abs().mean()
print("MAE=", MAE)
RMSE = math.sqrt((fouten**2).mean())
print("RMSE=", RMSE)
MAPE = (fouten/opbrengsten).abs().mean()
print("MAPE=", MAPE)

we kunnen weer functies maken:

In [ ]:
def mae(past, forecast_function):
  forecasted = forecast_past(past, forecast_function)
  errors = pd.Series(forecasted) - past
  return errors.abs().mean()

def rmse(past, forecast_function):
  forecasted = forecast_past(past, forecast_function)
  errors = pd.Series(forecasted) - past
  return math.sqrt((errors**2).mean())

def mape(past, forecast_function):
  forecasted = forecast_past(past, forecast_function)
  errors = pd.Series(forecasted) - past
  return (errors / past).abs().mean()

In [ ]:
print("mae = ", mae(opbrengsten, naive_forecasting))
print("rmse = ", rmse(opbrengsten, naive_forecasting))
print("mape = ", mape(opbrengsten, naive_forecasting))

In [ ]:
print("mae = ", mae(opbrengsten, average_forecasting))
print("rmse = ", rmse(opbrengsten, average_forecasting))
print("mape = ", mape(opbrengsten, average_forecasting))

In [ ]:
print("mae = ", mae(opbrengsten, moving_average_forecasting(4)))
print("rmse = ", rmse(opbrengsten, moving_average_forecasting(4)))
print("mape = ", mape(opbrengsten, moving_average_forecasting(4)))

In [ ]:
print("mae = ", mae(opbrengsten, lineair_combination_forecasting(4)))
print("rmse = ", rmse(opbrengsten, lineair_combination_forecasting(4)))
print("mape = ", mape(opbrengsten, lineair_combination_forecasting(4)))

# Trend forecasting

In [ ]:
# functies voor regressie
def general_regression(x:pd.Series, y:pd.Series, degree=1, exp=False, log=False):
    fun_y = inv_fun_y = lambda x:x
    fun_x = lambda x:x
    if (exp):
        fun_y = np.exp
        inv_fun_y = np.log
    if (log):
        fun_x = np.log
    model = np.polyfit(fun_x(x), inv_fun_y(y), degree)
    line = np.poly1d(model)
    predict = lambda x:fun_y(line(fun_x(x)))
    predicted = predict(x)
    squared_residuals = (predicted-y)**2
    variance_y = (y-y.mean())**2
    se = math.sqrt(squared_residuals.mean())
    r2 = 1 - (squared_residuals.sum() / variance_y.sum())
    result = [se, r2, predict] + [model[-i] for i in range(1, len(model)+1)]
    index = ["se", "r2", "predict"] + [chr(i+96) for i in range(1, len(model)+1)]
    return pd.Series(result, index=index)

def plot_regressionline(ax, reg_result, min, max, linecol="red", plot_error=False, errorcol="#FFFF0080"):
    se = reg_result.se
    x = np.arange(min, max, (max-min)/100)
    y = reg_result.predict(x)
    if plot_error:
        ax.fill_between(x, y-se, y+se, color=errorcol)
    ax.plot(x, y, color=linecol)

In [ ]:
reg = general_regression(pd.Series(range(n)), pd.Series(opbrengsten))
print(reg)

In [ ]:
def trend_forecasting_model(past):
  n = len(past)
  x = pd.Series(range(n))
  y = pd.Series(past)
  reg = general_regression(x, y)
  return reg.predict

In [ ]:
mijnTrend = trend_forecasting_model(opbrengsten)
mijnTrend(20)

In [ ]:
voorspeld = mijnTrend(range(24))
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
plt.plot(range(24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Lineaire regressie")
# fig.show()

In [ ]:
voorspeld = pd.Series(mijnTrend(range(20)))
fouten = voorspeld - opbrengsten
MAE = fouten.abs().mean()
print("MAE=", MAE)
RMSE = math.sqrt((fouten**2).mean())
print("RMSE=", RMSE)
MAPE = (fouten/opbrengsten).abs().mean()
print("MAPE=", MAPE)

In [ ]:
# se van de regressie is gelijk aan de RMSE
reg = general_regression(pd.Series(range(20)), pd.Series(opbrengsten))
print("se=", reg.se)

In [ ]:
voorspeld = mijnTrend(range(20, 24))
print(voorspeld)

## Seasonal decomposition

### Seizoensgrootte bepalen

In [ ]:
fig, ax = plt.subplots()
pd.plotting.autocorrelation_plot(opbrengsten, ax=ax)
ax.set_xlim(0, 20) # dit kan handig zijn als er veel data is
ax.set_xlabel("offset")
ax.set_ylabel("correlatie")
ax.set_title("Auto-correlation opbrengsten")
# fig.show()

### Decompositie

In [ ]:
result = seasonal_decompose(opbrengsten, model='multiplicative', period=4)
print(result.trend)
print(result.seasonal)
print(result.resid)

In [ ]:
_ = result.plot()

In [ ]:
trend = result.trend
seizoen = result.seasonal
ruis = result.resid

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(8,8))
ax1.set_title("Multiplicatieve decompositie")
ax1.plot(range(20), opbrengsten, "o-")
ax1.set_ylabel("observed")
ax2.plot(range(20), trend, "o-")
reg = general_regression(pd.Series(range(2,18)), pd.Series(result.trend[2:18]), exp=True)
plot_regressionline(ax2, reg, 0, 20)
ax2.set_ylabel("trend")
ax3.plot(range(20), seizoen*5, "o-")
ax3.set_ylabel("season")
ax4.scatter(range(20), ruis)
ax4.set_ylabel("residue")
ax4.set_xlabel("kwartaal")
# fig.show()

### Voorspellingen doen

In [ ]:
# zoek model voor de trend via regressie
# in dit geval geeft exponentiële regressie het beste resultaat
reg = general_regression(pd.Series(range(2,18)), pd.Series(result.trend[2:18]), exp=True)
print(reg)

In [ ]:
voorspeld = reg.predict(range(20, 24))*result.seasonal[0:4]
print(voorspeld)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20, 24), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Seasonal decomposition forecasting")
# fig.show()

In [ ]:
voorspeld = reg.predict(range(20)) * result.seasonal
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Seasonal decomposition forecasting")
# fig.show()

In [ ]:
# fouten berekenen
voorspeld = pd.Series(reg.predict(range(20))*result.seasonal)
fouten = voorspeld - opbrengsten
MAE = fouten.abs().mean()
print("MAE=", MAE)
RMSE = math.sqrt((fouten**2).mean())
print("RMSE=", RMSE)
MAPE = (fouten/opbrengsten).abs().mean()
print("MAPE=", MAPE)

In [ ]:
# additief beter???
result = seasonal_decompose(opbrengsten, model='additive', period=4)
reg = general_regression(pd.Series(range(2,18)), pd.Series(result.trend[2:18]), exp=True)
voorspeld = reg.predict(range(20)) + result.seasonal
fig, ax = plt.subplots()
ax.plot(range(20), opbrengsten, "o-", label="gegeven")
ax.plot(range(20), voorspeld, "^-", label="voorspeld")
ax.legend()
ax.set_xlabel("kwartaal")
ax.set_ylabel("opbrengst (EUR)")
ax.set_title("Seasonal decomposition forecasting")
# fig.show()
voorspeld = pd.Series(reg.predict(range(20)) + result.seasonal)
fouten = voorspeld - opbrengsten
MAE = fouten.abs().mean()
print("MAE=", MAE)
RMSE = math.sqrt((fouten**2).mean())
print("RMSE=", RMSE)
MAPE = (fouten/opbrengsten).abs().mean()
print("MAPE=", MAPE)